In [76]:
"""
Made by Joe Geoghegan
"""
import pandas as pd
from IPython.display import clear_output

### Helper Class

In [77]:
class wordleBreaker:
    """
    Made by Joe Geoghegan
    -Work in Progress
    When provided a dictionary, provides tools to solve the New York Times game Wordle
    """
    # Class Variables
    word_pool=None
    origin=None

    # Class creation and management
    def __init__(self, dict_url):
        self.origin = pd.read_csv(dict_url,index_col='words')
        self.word_pool = self.origin.copy()
        self.update_scores()
    def reset(self):
        self.word_pool = self.origin.copy()
        self.update_scores()

    # Dictionary Manipulation Functions
    def remove_letter_from_character(self,letter,character_space):
        """
        Creates a mask where True if a word DOES NOT contains the given letter at the given character space
        Applies mask to that character row which turns the values we want to remove to NaN
        Then drop the NaN rows
        """
        character = "c"+str(character_space)
        self.word_pool[character] = self.word_pool[character][self.word_pool[character]!=letter]
        self.word_pool.dropna(inplace=True)
    
    def not_included_letter(self,letter):
        """
        Runs the remove_letter_from_character operation on all character spaces
        """
        for char in range(1,6):
            self.remove_letter_from_character(letter,char)
    
    def set_letter_to_character(self,letter,character_space):
        """
        Filters the dictionary to only have words that would fit a GREEN case
        Creates a mask where True if a word DOES contains the given letter at the given character space
        Applies mask to that character row which turns the values we want to remove to NaN
        Then drop the NaN rows
        """
        character = "c"+str(character_space)
        self.word_pool[character] = self.word_pool[character][self.word_pool[character]==letter]
        self.word_pool.dropna(inplace=True)
    
    def filter_to_include_letter(self,letter):
        """
        Uses a mask for each character space that has True where given letter is present
        Creates a mask which combines all the given letter present masks
        """
        in_word = (
            (self.word_pool['c1']==letter) |
            (self.word_pool['c2']==letter) |
            (self.word_pool['c3']==letter) |
            (self.word_pool['c4']==letter) |
            (self.word_pool['c5']==letter)
            )
        self.word_pool = self.word_pool[in_word]
        self.word_pool.dropna(inplace=True)
    
    def yellow_filter(self, letter, character_space):
        """
        Filters the dictionary to only have words that would fit a YELLOW case
            Uses the filter_to_include_letter() and remove_letter_from_character() functions
        """
        self.filter_to_include_letter(letter)
        self.remove_letter_from_character(letter,character_space)

    # Guess Evaluation Functions
    def update_scores(self,sort=True):
        """
        Populates/updates the score value to a rudamentary point system
        The score is a sum of character scores for each character in a word
        The character score is the count of the letter's occurances in the
            entire word pool's population at that character spot
        """
        # Count all occurances of every letter in each character space
        values = pd.DataFrame({
            "c1":self.word_pool["c1"].value_counts(),
            "c2":self.word_pool["c2"].value_counts(),
            "c3":self.word_pool["c3"].value_counts(),
            "c4":self.word_pool["c4"].value_counts(),
            "c5":self.word_pool["c5"].value_counts()
        }).fillna(0)
        # For each word's letters add space scores together
        score_grid = pd.DataFrame(index=self.word_pool.index) # Create an empty df with the words as index
        for char in ['c1','c2','c3','c4','c5']: # for each character space
            score_col = pd.DataFrame(values[char][self.word_pool[char]] # lookup the spaces score for each word in the word pool
                            ).set_index(self.word_pool.index) # make it a dictionary with the words as index
            score_grid[f'{char}_score'] = score_col # Add the score column as a column in the score grid
        self.word_pool['score'] = score_grid.sum(axis='columns')
        if sort: # sorts by default, or if specifically selected
            self.sort_pool()
    
    def sort_pool(self):
        """
        Updates the word pool based on the score
        """
        self.word_pool.sort_values(by='score',ascending=False,inplace=True)

    def unique_letter_words(self):
        """
        Provides the dictionary sliced to only include words with all unique letters
        """
        c1Match = ( (self.word_pool['c1']!=self.word_pool['c2']) &
                    (self.word_pool['c1']!=self.word_pool['c3']) &
                    (self.word_pool['c1']!=self.word_pool['c4']) & 
                    (self.word_pool['c1']!=self.word_pool['c5']) )
        c2Match = ( (self.word_pool['c2']!=self.word_pool['c3']) &
                    (self.word_pool['c2']!=self.word_pool['c4']) &
                    (self.word_pool['c2']!=self.word_pool['c5']) )
        c3Match = ( (self.word_pool['c3']!=self.word_pool['c4']) &
                    (self.word_pool['c3']!=self.word_pool['c5']) )
        c4Match = ( (self.word_pool['c4']!=self.word_pool['c5']) )
        return self.word_pool[c1Match & c2Match & c3Match & c4Match]
    
    #play wordle functions
    def process_guess(self, guess, result):
        """
        Takes a guess and result and correctly calls the correct
        g/G means green, y/Y means yellow, b/B means black
        """
        for char in range(1,6):
            g_char = guess[char-1]
            char_result = result[char-1]
            if (char_result == 'g') or (char_result == 'G'): #green
                self.set_letter_to_character(g_char,char)
            elif (char_result == 'y') or (char_result == 'Y'): #yellow
                self.yellow_filter(g_char,char)
            elif (char_result == 'b') or (char_result == 'B'): #black
                self.not_included_letter(g_char)
            else:
                print(f"Whoops! g/G means green, y/Y means yellow, b/B means black.\nI did not recognize that letter!\nPlease manually redo character #{char}")
    
    def disp(self):
        """
        Displays only words and the score
        """
        return self.word_pool['score']

    def play(self):
        """
        Prompts guess, then result, then displays sorted words and repeats until result is all green
        """
        print("Welcome to Wordle Breaker!")
        self.update_scores()
        print("Top ten guesses:\n", self.disp().head(10))
        for turn in range(1,6):
            guess = input("Guess: ")
            print("for Results input:\n",
                "g/G means green\n",
                "y/Y means yellow\n"
                " b/B means black\n")
            result = input("Result: ")
            if result.lower() == 'ggggg':
                clear_output(wait=True)
                print("Glad I could help!")
                return
            else:
                clear_output(wait=True)
                self.process_guess(guess,result)
                self.update_scores()
                print("Top ten guesses:\n", self.disp().head(10))
        print("Sorry! Better luck next time!")

## Running

In [78]:
# Set dictionary path 
# dict_path = ".\Resources\dict_allAccepted.csv"
dict_path = ".\Resources\dict_minedAnswers.csv"
# CSV must be formatted as
# word,c1,c2,c3,c4,c5,score

In [79]:
# Initialize Wordle Breaker
wb = wordleBreaker(dict_path)

In [80]:
# wb.play()